In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from importlib import reload
from time import time
import pickle
import re

import numpy as np
import scipy as sc
import pandas as pd
import torch
print(torch.cuda.is_available())

import models
import dre
import rna
import vae
import designers
import utils

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams["font.size"] = 12

import seaborn as sns
sns.set_style('whitegrid')

True


2024-12-17 07:36:19.599056: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-17 07:36:19.599083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-17 07:36:19.600269: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 07:36:19.607169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-17 07:36:21.299209: W tensorflow/comp

In [2]:
# cannot contain any groups that don't have any data from name2designdata
# MDRE.fit() will throw error
mdre_group_regex_strs = [
    'adalead-ridge-0.\d',
    'adalead-ff-0.\d',
    'adalead-cnn-0.\d',
    'biswas-ridge-0.\d',
    'biswas-ff-0.\d',
    'biswas-cnn-0.\d',
    'dbas-ff-0.\d',
    'dbas-cnn-0.\d',
    'pex-ridge',
    'pex-ff',
    'pex-cnn',
    'dbas-ridge-0.1t\d',
    'dbas-ridge-0.2t\d',
    'cbas-ridge-0.1t\d',
    'cbas-ridge-0.2t\d',
    'cbas-ridge-0.3t\d',
    'cbas-ridge-0.4t\d',
    'cbas-ridge-0.5t\d',
    'cbas-ridge-0.6t\d',
    'cbas-ridge-0.7t\d',
    'cbas-ridge-0.8t\d',
    'cbas-ridge-0.9t\d',
    'cbas-ff-0.1',
    'cbas-ff-0.2',
    'cbas-ff-0.3',
    'cbas-ff-0.4',
    'cbas-ff-0.5',
    'cbas-ff-0.6',
    'cbas-ff-0.7',
    'cbas-ff-0.8',
    'cbas-ff-0.9',
    'cbas-cnn-0.1',
    'cbas-cnn-0.2',
    'cbas-cnn-0.3',
    'cbas-cnn-0.4',
    'cbas-cnn-0.5',
    'cbas-cnn-0.6',
    'cbas-cnn-0.7',
    'cbas-cnn-0.8',
    'cbas-cnn-0.9',
    'vae-ridge'
]

In [ ]:
reload(dre)
reload(utils)
reload(rna)

# ----- build menu of design algorithm names -----
adalead_thresholds = [0.2, 0.15, 0.1, 0.05, 0.01] # [0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1, 0.05, 0.01]
biswas_temperatures = [0.02, 0.015, 0.01, 0.005]
cbas_dbas_quantiles = np.arange(0.1, 0.91, 0.1)
dbas_ridge_quantiles = [0.1, 0.2]

design_names = ['vae-ridge']
# cbas ridge
for q in cbas_dbas_quantiles:
    q = round(q, 2)
    design_names.append(f'cbas-ridge-{q}t19')

# dbas ridge
for q in dbas_ridge_quantiles:
    q = round(q, 2)
    design_names.append(f'dbas-ridge-{q}t19')
    
for model_name in ['ridge', 'ff', 'cnn']:
    
    # adalead
    for thresh in adalead_thresholds:
        thresh = round(thresh, 4)
        design_names.append(f'adalead-{model_name}-{thresh}')
    
    # biswas
    for temp in biswas_temperatures:
        temp = round(temp, 4)
        design_names.append(f'biswas-{model_name}-{temp}') 
    
    # PEX
    design_names.append(f'pex-{model_name}') 
        
    if model_name != 'ridge':
        # cbas
        for q in cbas_dbas_quantiles:
            q = round(q, 2)
            design_names.append(f'cbas-{model_name}-{q}')
        
        # dbas
        for q in cbas_dbas_quantiles:
            q = round(q, 2)
            design_names.append(f'dbas-{model_name}-{q}')
    

design_pkl_fname = '/data/wongfanc/rna-results/designs-041124.pkl'
model_and_data_fname_no_ftype = 'nsd0.02-040324'
cal_pkl_fname = '/data/wongfanc/rna-data/calibration-pm0.08-nsd0.02_041524_2.pkl'
target_values = np.arange(0.25, 0.501, 0.001)
n_trial = 200
n_mdre_hidden = 128
n_mdre_epoch = 100
use_quadratic_layer_mdre = True
n_forecast_designs = 1000
pp_csv_fname = '/data/wongfanc/rna-results/selection-pp-121624-2.csv'
cp_csv_fname = '/data/wongfanc/rna-results/selection-cp-121624-2.csv'
device = torch.device('cuda')

df = rna.run_pp_selection_experiments(
    design_names,
    design_pkl_fname,
    model_and_data_fname_no_ftype,
    cal_pkl_fname,
    mdre_group_regex_strs,
    target_values,
    n_trial,
    n_forecast_designs=n_forecast_designs,
    use_quadratic_layer_mdre=use_quadratic_layer_mdre,
    n_mdre_hidden=n_mdre_hidden,
    n_mdre_epoch=n_mdre_epoch,
    pp_csv_fname=pp_csv_fname,
    cp_csv_fname=cp_csv_fname,
    device=device,
)

No design data for dbas-ridge-0.3t0, exiting MDRE select_intermediate_iterations.
No design data for dbas-ridge-0.4t0, exiting MDRE select_intermediate_iterations.
No design data for dbas-ridge-0.5t0, exiting MDRE select_intermediate_iterations.
No design data for dbas-ridge-0.6t0, exiting MDRE select_intermediate_iterations.
No design data for dbas-ridge-0.7t0, exiting MDRE select_intermediate_iterations.
No design data for dbas-ridge-0.8t0, exiting MDRE select_intermediate_iterations.
No design data for dbas-ridge-0.9t0, exiting MDRE select_intermediate_iterations.
All design names in provided design data:
cbas-ridge-0.1t19
cbas-ridge-0.2t19
cbas-ridge-0.3t19
cbas-ridge-0.4t19
cbas-ridge-0.5t19
cbas-ridge-0.6t19
cbas-ridge-0.7t19
cbas-ridge-0.8t19
cbas-ridge-0.9t19
dbas-ridge-0.1t19
dbas-ridge-0.2t19
adalead-ridge-0.5
adalead-ridge-0.4
adalead-ridge-0.3
adalead-ridge-0.2
adalead-ridge-0.1
adalead-ridge-0.05
adalead-ridge-0.01
pex-ridge
adalead-ff-0.5
adalead-ff-0.4
adalead-ff-0.3
ada

100%|████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.96it/s]


Min train loss 0.72, min val loss 0.68

Fitting MDRE for adalead-ff-0.\d, which has 5 design algorithms:
train
adalead-ff-0.2
adalead-ff-0.1
adalead-ff-0.05
adalead-ff-0.01
adalead-ff-0.15
One-hot-encoding all 6 categories of sequences...
  Done. (4 s)


100%|████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.28it/s]


Min train loss 0.66, min val loss 0.62

Fitting MDRE for adalead-cnn-0.\d, which has 5 design algorithms:
train
adalead-cnn-0.2
adalead-cnn-0.1
adalead-cnn-0.05
adalead-cnn-0.01
adalead-cnn-0.15
One-hot-encoding all 6 categories of sequences...
  Done. (3 s)


...

dbas-cnn-0.3 has LBs 0.2859, 0.3167
dbas-cnn-0.4 has LBs 0.3021, 0.3262
dbas-cnn-0.5 has LBs 0.3003, 0.3484
dbas-cnn-0.6 has LBs 0.2865, 0.3173
dbas-cnn-0.7 has LBs 0.3343, 0.3366
dbas-cnn-0.8 has LBs 0.3269, 0.3269
dbas-cnn-0.9 has LBs 0.3186, 0.3235
Done running 55 / 200 trials (14375 s).
Saved to /data/wongfanc/rna-results/selection-pp-121624-2.csv and /data/wongfanc/rna-results/selection-cp-121624-2.csv (14377 s).

vae-ridge has LBs 0.3118, 0.3118
cbas-ridge-0.1t19 has LBs 0.2818, 0.2818
cbas-ridge-0.2t19 has LBs 0.3024, 0.3024
cbas-ridge-0.3t19 has LBs 0.2765, 0.2976
cbas-ridge-0.4t19 has LBs 0.2698, 0.2698
cbas-ridge-0.5t19 has LBs 0.2903, 0.2903
cbas-ridge-0.6t19 has LBs 0.3148, 0.3148
cbas-ridge-0.7t19 has LBs 0.2901, 0.2901
cbas-ridge-0.8t19 has LBs 0.3209, 0.3209
cbas-ridge-0.9t19 has LBs 0.4165, 0.4386
dbas-ridge-0.1t19 has LBs 0.2770, 0.3252
dbas-ridge-0.2t19 has LBs 0.3923, 0.3923
adalead-ridge-0.2 has LBs 0.3198, 0.3198
adalead-ridge-0.15 has LBs 0.3741, 0.3978
adalead-ri

dbas-ff-0.5 has LBs 0.3601, 0.3601
dbas-ff-0.6 has LBs 0.3478, 0.3478
dbas-ff-0.7 has LBs 0.3723, 0.3723
dbas-ff-0.8 has LBs 0.3462, 0.3462
dbas-ff-0.9 has LBs 0.3636, 0.3636
adalead-cnn-0.2 has LBs 0.3047, 0.3047
adalead-cnn-0.15 has LBs 0.3098, 0.3098
adalead-cnn-0.1 has LBs 0.3170, 0.3861
adalead-cnn-0.05 has LBs 0.3651, 0.4212
adalead-cnn-0.01 has LBs 0.3582, 0.3812
biswas-cnn-0.02 has LBs 0.3121, 0.3244
biswas-cnn-0.015 has LBs 0.3014, 0.3188
biswas-cnn-0.01 has LBs 0.3732, 0.3732
biswas-cnn-0.005 has LBs 0.3941, 0.3941
pex-cnn has LBs 0.3668, 0.3668
cbas-cnn-0.1 has LBs 0.2682, 0.2682
cbas-cnn-0.2 has LBs 0.2972, 0.2972
cbas-cnn-0.3 has LBs 0.3099, 0.3099
cbas-cnn-0.4 has LBs 0.2782, 0.2782
cbas-cnn-0.5 has LBs 0.2787, 0.2787
cbas-cnn-0.6 has LBs 0.2642, 0.3363
cbas-cnn-0.7 has LBs 0.2809, 0.3014
cbas-cnn-0.8 has LBs 0.2961, 0.2961
cbas-cnn-0.9 has LBs 0.2461, 0.2461
dbas-cnn-0.1 has LBs 0.3915, 0.3915
dbas-cnn-0.2 has LBs 0.2984, 0.3088
dbas-cnn-0.3 has LBs 0.2979, 0.3565
dbas-c

biswas-ridge-0.02 has LBs 0.3605, 0.3605
biswas-ridge-0.015 has LBs 0.3062, 0.3062
biswas-ridge-0.01 has LBs 0.3502, 0.3762
biswas-ridge-0.005 has LBs 0.3432, 0.3538
pex-ridge has LBs 0.2964, 0.3203
adalead-ff-0.2 has LBs 0.3391, 0.3538
adalead-ff-0.15 has LBs 0.3470, 0.3608
adalead-ff-0.1 has LBs 0.3703, 0.3713
adalead-ff-0.05 has LBs 0.3814, 0.4023
adalead-ff-0.01 has LBs 0.3835, 0.3835
biswas-ff-0.02 has LBs 0.2801, 0.3264
biswas-ff-0.015 has LBs 0.2912, 0.3374
biswas-ff-0.01 has LBs 0.3074, 0.3308
biswas-ff-0.005 has LBs 0.3398, 0.3398
pex-ff has LBs 0.3157, 0.3499
cbas-ff-0.1 has LBs 0.2895, 0.2895
cbas-ff-0.2 has LBs 0.3033, 0.3033
cbas-ff-0.3 has LBs 0.2689, 0.2689
cbas-ff-0.4 has LBs 0.2961, 0.2961
cbas-ff-0.5 has LBs 0.3989, 0.3989
cbas-ff-0.7 has LBs 0.3230, 0.3230
cbas-ff-0.8 has LBs 0.3497, 0.3497
cbas-ff-0.9 has LBs 0.3321, 0.3321
dbas-ff-0.1 has LBs 0.2929, 0.2929
dbas-ff-0.2 has LBs 0.3236, 0.3236
dbas-ff-0.3 has LBs 0.3153, 0.3153
dbas-ff-0.4 has LBs 0.2911, 0.3189
dbas

vae-ridge has LBs 0.3060, 0.3060
cbas-ridge-0.1t19 has LBs 0.2265, 0.2614
cbas-ridge-0.2t19 has LBs 0.2899, 0.2899
cbas-ridge-0.3t19 has LBs 0.2628, 0.2628
cbas-ridge-0.4t19 has LBs 0.2384, 0.2558
cbas-ridge-0.5t19 has LBs 0.3292, 0.3292
cbas-ridge-0.6t19 has LBs 0.3085, 0.3085
cbas-ridge-0.7t19 has LBs 0.3521, 0.3521
cbas-ridge-0.8t19 has LBs 0.3843, 0.3843
cbas-ridge-0.9t19 has LBs 0.3853, 0.3853
dbas-ridge-0.1t19 has LBs 0.3329, 0.3329
dbas-ridge-0.2t19 has LBs 0.3393, 0.4307
adalead-ridge-0.2 has LBs 0.3308, 0.3308
adalead-ridge-0.15 has LBs 0.3400, 0.3400
adalead-ridge-0.1 has LBs 0.3477, 0.3477
adalead-ridge-0.05 has LBs 0.3681, 0.3681
adalead-ridge-0.01 has LBs 0.3669, 0.3669
biswas-ridge-0.02 has LBs 0.3262, 0.3262
biswas-ridge-0.015 has LBs 0.3062, 0.3062
biswas-ridge-0.01 has LBs 0.3112, 0.3112
biswas-ridge-0.005 has LBs 0.3279, 0.3279
pex-ridge has LBs 0.2757, 0.2757
adalead-ff-0.2 has LBs 0.3251, 0.3251
adalead-ff-0.15 has LBs 0.3351, 0.3351
adalead-ff-0.1 has LBs 0.3436, 0

cbas-cnn-0.1 has LBs 0.2980, 0.3145
cbas-cnn-0.2 has LBs 0.2980, 0.2980
cbas-cnn-0.3 has LBs 0.2886, 0.2886
cbas-cnn-0.4 has LBs 0.2939, 0.2939
cbas-cnn-0.5 has LBs 0.2589, 0.2589
cbas-cnn-0.6 has LBs 0.2726, 0.2726
cbas-cnn-0.7 has LBs 0.2600, 0.2600
cbas-cnn-0.8 has LBs 0.3080, 0.3080
cbas-cnn-0.9 has LBs 0.3008, 0.3008
dbas-cnn-0.1 has LBs 0.3213, 0.3213
dbas-cnn-0.2 has LBs 0.3220, 0.3220
dbas-cnn-0.3 has LBs 0.2873, 0.2873
dbas-cnn-0.4 has LBs 0.2839, 0.2917
dbas-cnn-0.5 has LBs 0.2965, 0.3179
dbas-cnn-0.6 has LBs 0.3036, 0.3065
dbas-cnn-0.7 has LBs 0.2969, 0.2975
dbas-cnn-0.8 has LBs 0.3022, 0.3022
dbas-cnn-0.9 has LBs 0.3292, 0.3292
Done running 66 / 200 trials (17253 s).
Saved to /data/wongfanc/rna-results/selection-pp-121624-2.csv and /data/wongfanc/rna-results/selection-cp-121624-2.csv (17256 s).

vae-ridge has LBs 0.2682, 0.2682
cbas-ridge-0.1t19 has LBs 0.3351, 0.3351
cbas-ridge-0.2t19 has LBs 0.2824, 0.2824
cbas-ridge-0.3t19 has LBs 0.3110, 0.3110
cbas-ridge-0.4t19 has LBs